# Generate a mosaic of the GOES-16 channels in Satpy and ImageMagick

In [ ]:
import glob
from satpy.scene import Scene

scn = Scene(
    filenames=glob.glob("/path/to/the/Goes-16/data/*RadF*"),
    reader='abi_l1b'
)

channels = ['C{channel:02d}'.format(channel=chn) for chn in range(1, 17)]
scn.load(channels)
scn.save_datasets(filename='{name}.png')

Then we can use the ImageMagick command `montage` to join all the images together.

In [ ]:
!montage C??.png -geometry 512x512+4+4 -background black montage_abi.jpg

<img src="montage_abi.jpg">